# Average Service

Since each service runs in a separate python process, we will write the python code to a file, and **expect you to run the script from a terminal**.

In [1]:
# Configure python path to load incubator modules
import sys
import os

# Get the current working directory. Should be 1-Incubator-Service
current_dir = os.getcwd()

assert os.path.basename(current_dir) == '1-Incubator-Service', 'Current directory is not 1-Incubator-Service'

# Get the parent directory. Should be the root of the repository
parent_dir = os.path.dirname(current_dir)

# The root of the repo should contain the incubator_dt folder. Otherwise something went wrong in 0-Pre-requisites.
assert os.path.exists(os.path.join(parent_dir, 'incubator_dt')), 'incubator_dt folder not found in the repository root'

incubator_dt_software_dir = os.path.join(parent_dir, 'incubator_dt', 'software')

assert os.path.exists(incubator_dt_software_dir), 'incubator_dt software directory not found'

# Add the parent directory to sys.path
sys.path.append(incubator_dt_software_dir)

In [2]:
%%writefile average_service.py

# Configure python path to load incubator modules
import sys
import os
import logging
import logging.config
import time

# Get the current working directory. Should be 1-Incubator-Service
current_dir = os.getcwd()

assert os.path.basename(current_dir) == '1-Incubator-Service', 'Current directory is not 1-Incubator-Service'

# Get the parent directory. Should be the root of the repository
parent_dir = os.path.dirname(current_dir)

# The root of the repo should contain the incubator_dt folder. Otherwise something went wrong in 0-Pre-requisites.
assert os.path.exists(os.path.join(parent_dir, 'incubator_dt')), 'incubator_dt folder not found in the repository root'

incubator_dt_software_dir = os.path.join(parent_dir, 'incubator_dt', 'software')

assert os.path.exists(incubator_dt_software_dir), 'incubator_dt software directory not found'

# Add the parent directory to sys.path
sys.path.append(incubator_dt_software_dir)

from incubator.communication.server.rpc_server import RPCServer

class AverageService(RPCServer):
    """
    This is a server service that computes the average of a given list of values.
    It extends the RPCServer class, which is a class that listens to a RabbitMQ queue and waits for messages to arrive, and hides much of the complexity of the server service. 
    All we need to do to implement the average service is implement a method called "compute_average" that takes a list of values and returns the average of those values. This method will be called by the RPCServer class when a message arrives in the RabbitMQ queue containing the name of the method to call and the arguments to pass to the method.
    """
    def __init__(self, rabbitmq_config):
        super().__init__(**rabbitmq_config)
        self._l = logging.getLogger("AverageService")

    def setup(self):
        """ 
        Setup the RabbitMQ connection and declare the routing_key (this is the topic that this server will listen to) and queue (the name of the queue where all messages addressed to routing_key will be placed in by the RabbitMQ server).

        We use the same name for both the routing_key and the queue name. This is not necessary, but it makes it easier to understand what is happening in the RabbitMQ server.        
        """
        super(AverageService, self).setup(routing_key='incubator.dtcourse.average_service', queue_name='incubator.dtcourse.average_service')

        self._l.info(f"AverageService setup complete.")

    def compute_average(self, values, reply_fun):
        """ 
        This is the method that will be invoked by the RPCServer class when a message arrives in the RabbitMQ queue. The reply_fun is a function that we can call to send the results back to the client that sent the message.
        """
        average = 0.0

        # Log the values received.
        self._l.info(f"compute_average called. Received values: {values}")

        # Compute the average of the values.
        if len(values) > 0:
            average = sum(values) / len(values)
        else:
            self._l.warning("Received an empty list of values. Cannot compute average. Returning error")
            reply_fun({"error": "Received an empty list of values. Cannot compute average."})
            return

        # Prepare the results to send back.
        result_msg = {
            "average": average
        }

        # Send results back.
        reply_fun(result_msg)
    
if __name__ == "__main__":
    # Get utility functions to config logging and load configuration
    from incubator.config.config import load_config
    from pyhocon import ConfigFactory
    # Configure logging level to info
    logging.basicConfig(level=logging.INFO)
    # Get path to the startup.conf file used in the incubator dt:
    startup_conf = os.path.join(os.path.dirname(os.getcwd()), 'incubator_dt', 'software','startup.conf')
    assert os.path.exists(startup_conf), 'startup.conf file not found'

    # The startup.conf comes from the incubator dt repository.
    config = ConfigFactory.parse_file(startup_conf)
    service = AverageService(rabbitmq_config=config["rabbitmq"])

    service.setup()
    
    # Start the AverageService
    service.start_serving()

Overwriting average_service.py


Run the command below in a new terminal windows to start the service

```bash
python average_service.py
```

Now that the service is running let us send a request message to execute an operation.

In [3]:
# Import RPCClient class from incubator, which makes connecting to RabbitMQ and calling remote methods easy.
from incubator.communication.server.rpc_client import RPCClient
from pyhocon import ConfigFactory

# Get config
startup_conf = os.path.join(os.path.dirname(os.getcwd()), 'incubator_dt', 'software','startup.conf')
config = ConfigFactory.parse_file(startup_conf)

with RPCClient(**(config["rabbitmq"])) as client:
    reply = client.invoke_method("incubator.dtcourse.average_service", "compute_average", {"values": [1.0, 2.0, 3.0, 4.0, 5.0]})
    print(reply)
    assert 2.9 < reply["average"] < 3.1, "Average is not 3.0"

{'average': 3.0}


You should see something like:
```powershell
PS C:\work\github\IncubatorDTCourse\1-Incubator-Service> python .\average_service.py
INFO:pika.adapters.utils.connection_workflow:Pika version 1.3.2 connecting to ('::1', 5672, 0, 0)
INFO:pika.adapters.utils.io_services_utils:Socket connected: <socket.socket fd=556, family=23, type=1, proto=6, laddr=('::1', 3823, 0, 0), raddr=('::1', 5672, 0, 0)>
INFO:pika.adapters.utils.connection_workflow:Streaming transport linked up: (<pika.adapters.utils.io_services_utils._AsyncPlaintextTransport object at 0x000002DB85E4B890>, _StreamingProtocolShim: <SelectConnection PROTOCOL transport=<pika.adapters.utils.io_services_utils._AsyncPlaintextTransport object at 0x000002DB85E4B890> params=<ConnectionParameters host=localhost port=5672 virtual_host=/ ssl=False>>).
INFO:pika.adapters.utils.connection_workflow:AMQPConnector - reporting success: <SelectConnection OPEN transport=<pika.adapters.utils.io_services_utils._AsyncPlaintextTransport object at 0x000002DB85E4B890> params=<ConnectionParameters host=localhost port=5672 virtual_host=/ ssl=False>>
INFO:pika.adapters.utils.connection_workflow:AMQPConnectionWorkflow - reporting success: <SelectConnection OPEN transport=<pika.adapters.utils.io_services_utils._AsyncPlaintextTransport object at 0x000002DB85E4B890> params=<ConnectionParameters host=localhost port=5672 virtual_host=/ ssl=False>>
INFO:pika.adapters.blocking_connection:Connection workflow succeeded: <SelectConnection OPEN transport=<pika.adapters.utils.io_services_utils._AsyncPlaintextTransport object at 0x000002DB85E4B890> params=<ConnectionParameters host=localhost port=5672 virtual_host=/ ssl=False>>
INFO:pika.adapters.blocking_connection:Created channel=1
INFO:AverageService:AverageService setup complete.
INFO:AverageService:compute_average called. Received values: [1.0, 2.0, 3.0, 4.0, 5.0]
```

In [4]:
# Now send something non sensical, to get an error:
with RPCClient(**(config["rabbitmq"])) as client:
    reply = client.invoke_method("incubator.dtcourse.average_service", "compute_average", {"values": []})
    print(reply)
    assert "error" in reply, "Error not received"

{'error': 'Received an empty list of values. Cannot compute average.'}


In [5]:
with RPCClient(**(config["rabbitmq"])) as client:
    reply = client.invoke_method("incubator.dtcourse.average_service", "some_other_method", {"values": []})
    print(reply)
    assert "error" in reply, "Error not received"

{'error': 'Method specified does not exist: some_other_method.'}


The output should now contain something like:
```powershell
INFO:AverageService:compute_average called. Received values: []
WARNING:AverageService:Received an empty list of values. Cannot compute average. Returning error
```

Now **terminate the average service process** by typing `Ctrl+c` on the terminal where you started the average service.

## Exercises

1. Change the average service so that, in addition to the average returned, it also returns the [standard error](https://en.wikipedia.org/wiki/Standard_error). The new output message should look like:
    ```python
    result_msg = {
        "average": average,
        "std_error": standard_error
    }
    ```
    Don't forget to test the service.
